In [2]:
import os
import sys
import shutil

import tensorflow as tf
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model import utils
from tensorflow.python.util import compat

In [12]:
# Remove flag names
from absl import flags

for name in list(flags.FLAGS):
    delattr(flags.FLAGS, name)

In [4]:
# simulate data

import numpy as np
np.random.seed(42)

# simulate dataset
age_ = np.random.normal(loc=25, scale=10, size=[1000, 1])
years_ = np.random.gamma(shape=9.0, scale=2.0, size=[1000, 1])
income_ = np.random.gamma(shape=3.0, scale=2.0, size=[1000, 1])
trad_ = np.random.weibull(a=2, size=[1000, 1])
num_friends = np.random.poisson(lam=3.5, size=[1000, 1])
other_ = np.random.normal(loc=5, scale=2, size=[1000, 1])

X = np.hstack((age_, years_, income_, 
     trad_, num_friends, other_))
y = np.random.normal(loc=500, scale=35, size=[1000, 1])

In [13]:
# Command line arguments
tf.app.flags.DEFINE_integer('training_iteration', 20,
                           'number of training iterations.')

tf.app.flags.DEFINE_integer('model_version', 1, 'version number of the model.')

tf.app.flags.DEFINE_string('checkpoint_dir', 
                           './checkpoints', 
                           'Directory for storing training checkpints.')

tf.app.flags.DEFINE_string('output_dir', './reg_export',
                         'Directory for storing exported model.')

FLAGS = tf.app.flags.FLAGS

In [7]:
sys.argv

['C:\\Users\\jg568_000\\Anaconda3\\envs\\tensorflow\\lib\\site-packages\\ipykernel_launcher.py',
 '-f',
 'C:\\Users\\jg568_000\\AppData\\Roaming\\jupyter\\runtime\\kernel-5d1993f4-5183-4937-b8a1-dccce5edc329.json']

In [10]:
def build_model(feature_names = []):
    
    for n in feature_names:
        if not isinstance(n, str):
            print('Feature names must be strings')
    
    a = tf.feature_column.numeric_column(feature_names[0])
    b = tf.feature_column.numeric_column(feature_names[1])
    c = tf.feature_column.numeric_column(feature_names[2])
    d = tf.feature_column.numeric_column(feature_names[3])
    e = tf.feature_column.numeric_column(feature_names[4])
    f = tf.feature_column.numeric_column(feature_names[5])

    feature_columns = [a, b, c, d, e, f]
    
    model = tf.estimator.LinearRegressor(feature_columns=feature_columns,
                                     optimizer=tf.train.AdamOptimizer(learning_rate=0.01))
    return model

In [ ]:
def main(_):
    print('Training...')
    
    # Reset graph session to default
    with tf.Graph().as_default():
        serialized_tf_example = tf.placeholder(tf.string, name='input_data')
        feature_configs = {'x': tf.placeholder(shape=[], dtype=tf.float32), }
    
        tf_example = tf.parse_example(serialized=serialized_tf_example, 
                                  features=feature_configs)
        
        # assign placeholder name for graph??
        x = tf.identity(input=tf_example['x'], name='x')
        
        # assign placeholder for output vairiable
        y_ = tf.placeholder(dtype='float', shape=[None,1])
        
        # Linear regression model from estimator
        net = build_model(feature_names=['age','income','years','bin','num','o'])
        
        # create saver 
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
            saver.restore(sess, ckpt.model_checkpoint_path)
            export_path = os.path.join(
            tf.compat.as_bytes(FLAGS.output_dir),
            tf.compat.as_bytes(str(FLAGS.model_version)))
            
            if os.path.exists(export_path):
                shutil.rmtree(export_path)
            
            # create model builder - builds graph ans session from export
            builder = tf.saved_model.builder.SavedModelBuilder(export_path)
            
            
        
        